<a href="https://colab.research.google.com/github/SW26010/Texture-Map-Toolbox/blob/main/metallic_smoothness_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image

# --- 用户预设参数 ---

# 1. 定义输入和输出文件路径
input_image_path = '/content/MobiusBracelet-color.png' # 请替换为您的蒙版图片路径 (确保它有Alpha通道)
ms_output_path = 'metallic_smoothness_map.png' # 金属度/光滑度贴图的保存路径
color_output_path = 'color_map.png' # 颜色贴图的保存路径

# 2. 为R, G, B三个通道代表的材质设定属性
# 数值范围: 0.0 - 1.0
# R通道代表金缮
mat_r_metallic = 1.0
mat_r_smoothness = 0.445
mat_r_color_hex = '#D69687FF'

# G通道代表陶瓷
mat_g_metallic = 0.0
mat_g_smoothness = 1.0
mat_g_color_hex = '#FBFFFCFF'

# B通道代表黑锆金
mat_b_metallic = 0.0
mat_b_smoothness = 0.727
mat_b_color_hex = '#393938FF'

# 3. 判定阈值 (0-255)
threshold = 127

# --- 代码实现 ---

# A. 转换金属度/光滑度值为0-255整数
m_r_255 = int(mat_r_metallic * 255)
s_r_255 = int(mat_r_smoothness * 255)
m_g_255 = int(mat_g_metallic * 255)
s_g_255 = int(mat_g_smoothness * 255)
m_b_255 = int(mat_b_metallic * 255)
s_b_255 = int(mat_b_smoothness * 255)

# B. 解析十六进制颜色值为RGBA元组 (0-255)
def hex_to_rgba(hex_color: str):
    """将#RRGGBBAA格式的字符串转换为(R, G, B, A)元组"""
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4, 6))

color_r_rgba = hex_to_rgba(mat_r_color_hex)
color_g_rgba = hex_to_rgba(mat_g_color_hex)
color_b_rgba = hex_to_rgba(mat_b_color_hex)


# C. 读取输入图片并转换为NumPy数组
try:
    # **修改点**: 将 .convert('RGB') 改为 .convert('RGBA') 以读取Alpha通道
    input_image = Image.open(input_image_path).convert('RGBA')
except FileNotFoundError:
    print(f"错误：找不到输入文件 '{input_image_path}'。请检查路径。")
    raise
input_array = np.array(input_image)

# 检查输入图片是否确实有Alpha通道
if input_array.shape[2] != 4:
    raise ValueError("错误：输入图片不包含Alpha通道。请提供带有透明度信息的图片。")


# D. 创建两个输出数组 (金属度/光滑度 + 颜色)
shape = (input_array.shape[0], input_array.shape[1], 4)
ms_output_array = np.zeros(shape, dtype=np.uint8)
color_output_array = np.zeros(shape, dtype=np.uint8)


# --- 核心处理逻辑 (生成蒙版并应用) ---

# 1. 计算出每个材质的蒙版区域
mask_r = input_array[:, :, 0] > threshold
mask_g = input_array[:, :, 1] > threshold
mask_b = input_array[:, :, 2] > threshold

# 2. 应用于金属度/光滑度贴图 (R/G通道)
ms_output_array[mask_r, 0] = m_r_255
ms_output_array[mask_r, 1] = s_r_255
ms_output_array[mask_g, 0] = m_g_255
ms_output_array[mask_g, 1] = s_g_255
ms_output_array[mask_b, 0] = m_b_255
ms_output_array[mask_b, 1] = s_b_255

# 3. 应用于颜色贴图 (RGB通道)
color_output_array[mask_r] = color_r_rgba
color_output_array[mask_g] = color_g_rgba
color_output_array[mask_b] = color_b_rgba


# --- 最终处理Alpha通道 ---

# **修改点**: 直接从输入数组复制Alpha通道到两个输出数组
source_alpha_channel = input_array[:, :, 3]
ms_output_array[:, :, 3] = source_alpha_channel
color_output_array[:, :, 3] = source_alpha_channel


# --- 保存输出文件 ---

# 1. 保存金属度/光滑度贴图
ms_output_image = Image.fromarray(ms_output_array)
ms_output_image.save(ms_output_path)
print(f"成功生成金属度/光滑度贴图，已保存至: {ms_output_path}")

# 2. 保存颜色贴图
color_output_image = Image.fromarray(color_output_array)
color_output_image.save(color_output_path)
print(f"成功生成颜色贴图，已保存至: {color_output_path}")

成功生成金属度/光滑度贴图，已保存至: metallic_smoothness_map.png
成功生成颜色贴图，已保存至: color_map.png
